# Решение СЛАУ с помощью метода простых итераций и метода Зейделя

In [10]:
import numpy as np

## Приведение матрицы к диагонально-доминирующему виду

In [11]:
def make_diagonally_dominant(A, b):
    """
    Преобразует систему A * x = b к диагонально-доминирующему виду,
    если это возможно. Возвращает новую матрицу A и вектор b.
    """
    n = len(A)
    new_A = np.zeros_like(A)
    new_b = np.zeros_like(b)
    remaining_rows = set(range(n))

    for col in range(n):
        max_row = -1
        max_val = -np.inf
        for row in remaining_rows:
            if abs(A[row, col]) > max_val:
                max_val = abs(A[row, col])
                max_row = row

        if max_row == -1:
            raise ValueError("Невозможно привести к диагонально-доминирующему виду.")
        
        new_A[col] = A[max_row]
        new_b[col] = b[max_row]
        remaining_rows.remove(max_row)

    for i in range(n):
        row_sum = np.sum(np.abs(new_A[i])) - np.abs(new_A[i, i])
        if np.abs(new_A[i, i]) <= row_sum:
            print("Внимание: не удалось достичь строгого диагонального преобладания.")

    return new_A, new_b


----

## Метод простых итераций

In [12]:
def simple_iteration_method(A, b, max_iter=1000, eps=1e-10, ensure_dominance=True):
    """
    Решает систему линейных уравнений методом простых итераций.
    """
    if ensure_dominance:
        A, b = make_diagonally_dominant(A, b)

    n = len(b)
    x = np.zeros(n)

    for k in range(max_iter):
        x_new = np.zeros(n)
        for i in range(n):
            s = np.dot(A[i, :], x) - A[i, i] * x[i]
            x_new[i] = (b[i] - s) / A[i, i]

        if np.linalg.norm(x_new - x) < eps:
            return x_new, k + 1

        x = x_new.copy()

    print("Достигнуто максимальное число итераций.")
    return x, max_iter


---

## Метод Зейделя

In [13]:
def seidel_method(A, b, eps=1e-10, max_iter=1000, ensure_dominance=True):
    """
    Решает систему линейных уравнений методом Зейделя.
    """
    if ensure_dominance:
        A, b = make_diagonally_dominant(A, b)

    n = len(A)
    x = np.zeros(n)

    for iteration in range(max_iter):
        x_prev = np.copy(x)

        for i in range(n):
            sigma = np.dot(A[i, :i], x[:i]) + np.dot(A[i, i + 1:], x[i + 1:])
            x[i] = (b[i] - sigma) / A[i, i]

        if np.linalg.norm(x - x_prev) < eps:
            return x, iteration + 1

    print(f"Достигнуто максимальное число итераций ({max_iter}).")
    return x, max_iter


---

## Пример использования

In [14]:
A = np.array([
    [2, 2, 10],
    [10, 1, 1],
    [2, 10, 1]
])

b = np.array([14, 12, 13])

print("Исходная матрица A:")
print(A)
print("Вектор правой части b:", b)

# Метод простых итераций
x_iter, iter_count = simple_iteration_method(A, b)
print("\nМетод простых итераций:")
print("Решение:", x_iter)
print("Количество итераций:", iter_count)
print("Проверка: A*x - b =", np.dot(A, x_iter) - b)

# Метод Зейделя
x_seidel, seidel_count = seidel_method(A, b)
print("\nМетод Зейделя:")
print("Решение:", x_seidel)
print("Количество итераций:", seidel_count)
print("Проверка: A*x - b =", np.dot(A, x_seidel) - b)


Исходная матрица A:
[[ 2  2 10]
 [10  1  1]
 [ 2 10  1]]
Вектор правой части b: [14 12 13]

Метод простых итераций:
Решение: [1. 1. 1.]
Количество итераций: 21
Проверка: A*x - b = [5.75468562e-11 3.62518904e-11 4.56754634e-11]

Метод Зейделя:
Решение: [1. 1. 1.]
Количество итераций: 9
Проверка: A*x - b = [ 1.77635684e-15 -2.82440737e-13  1.83852933e-11]
